In [7]:
from df.enhance import enhance, init_df, load_audio, save_audio
from pathlib import Path
from tqdm import tqdm
import glob
import os
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import torch
import soundfile as sf
import pyloudnorm as pyln
from scipy.io.wavfile import write
import librosa


OUTPUT_DIR = Path("train_denoised")

# make the OUTPUT_DIR if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
frcrn = pipeline(Tasks.acoustic_noise_suppression, model='speech_frcrn_ans_cirm_16k')
# Load default model
# Refer to: https://github.com/Rikorose/DeepFilterNet/blob/ebfc8a0b15dc02780788cd7786e52b4ed45aecd4/DeepFilterNet/df/enhance.py#L100
model, df_state, model_name_suffix = init_df(model_base_dir="DeepFilterNet3/")
# Get our SpeakerID audio
speakerID_audio_folder = "train"
audio_paths = glob.glob(f"{speakerID_audio_folder}/*.wav")
# Enhance each audio
for audio_path in tqdm(audio_paths):
    # get the audio filename without .wav extension
    audio_filename = os.path.splitext(os.path.basename(audio_path))[0]
    output_path = OUTPUT_DIR/f"{audio_filename}_enhanced.wav"
    frcrn(audio_path, output_path=output_path)

    data, rate = sf.read(output_path) # load audio
    # peak normalize audio to -0.1 dB as frcrn tend to output very soft
    peak_normalized_audio = pyln.normalize.peak(data, -0.1)  # not using loudness norm here as it causes a bit of clipping
    write(output_path, rate, peak_normalized_audio)

    audio, _ = load_audio(output_path, sr=df_state.sr())
    # Denoise the audio
    enhanced = enhance(model, df_state, audio)
    # Save for listening
    save_audio(output_path, enhanced, df_state.sr(), dtype=torch.float16)  # default is torch.int16 which causes clipping on some audios
    audio, sr = librosa.load(output_path, sr=22050)  # downsample to 22.05Khz
    sf.write(output_path, audio, sr)  # save the downsampled one


2023-06-12 12:19:43,280 - modelscope - INFO - initiate model from speech_frcrn_ans_cirm_16k
2023-06-12 12:19:43,282 - modelscope - INFO - initiate model from location speech_frcrn_ans_cirm_16k.
2023-06-12 12:19:43,292 - modelscope - INFO - initialize model from speech_frcrn_ans_cirm_16k
2023-06-12 12:19:43,911 - modelscope - WARNING - No preprocessor field found in cfg.
2023-06-12 12:19:43,912 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-06-12 12:19:43,912 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'speech_frcrn_ans_cirm_16k'}. trying to build by task and model information.
2023-06-12 12:19:43,912 - modelscope - WARNING - No preprocessor key ('speech_frcrn_ans_cirm_16k', 'acoustic-noise-suppression') found in PREPROCESSOR_MAP, skip building preprocessor.


2023-06-12 12:19:43 | INFO     | DF | Loading model settings of DeepFilterNet3
2023-06-12 12:19:43 | INFO     | DF | Initializing model `deepfilternet3`
2023-06-12 12:19:44 | INFO     | DF | Found checkpoint DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2023-06-12 12:19:44 | INFO     | DF | Running on device cuda:0
2023-06-12 12:19:44 | INFO     | DF | Model loaded


  0%|          | 0/32 [00:00<?, ?it/s]

inputs:(1, 240000)
padding: 24000
inputs after padding:(1, 264000)


/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
  3%|▎         | 1/32 [00:01<00:39,  1.26s/it]

...

/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
 94%|█████████▍| 30/32 [00:27<00:01,  1.10it/s]

inputs:(1, 240000)
padding: 24000
inputs after padding:(1, 264000)


/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
 97%|█████████▋| 31/32 [00:28<00:00,  1.10it/s]

inputs:(1, 240000)
padding: 24000
inputs after padding:(1, 264000)


/root/venvs/til2023/lib/python3.8/site-packages/torchaudio/functional/functional.py:1458: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  warnings.warn(
100%|██████████| 32/32 [00:29<00:00,  1.08it/s]
